In [1]:
import pdfplumber
import pandas as pd
import requests
import re
from fugashi import Tagger
from collections import Counter


In [49]:
def extract_text_from_pdf(pdf_path):
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

def perform_morphological_analysis(text):
    tagger = Tagger()
    words = tagger(text)
    word_list = [word.surface for word in words if re.match(r'\w', word.surface)]
    return words, word_list

def count_word_frequencies(word_list):
    word_counts = Counter(word_list)
    word_df = pd.DataFrame(word_counts.items(), columns=["単語", "頻度"]).sort_values(by="頻度", ascending=False)
    return word_df

def get_furigana(word, tagger):
    result = tagger.parse(word)
    return ''.join([entry.split('\t')[1] if len(entry.split('\t')) > 1 else word for entry in result.splitlines()])

def translate_to_english(word, deepl_api_key):
    url = "https://api-free.deepl.com/v2/translate"
    params = {
        "auth_key": deepl_api_key,
        "text": word,
        "target_lang": "EN"
    }
    response = requests.post(url, data=params)
    if response.status_code == 200:
        return response.json()["translations"][0]["text"]
    else:
        return "Error"

def perform_morphological_analysis(text):
    tagger = Tagger()
    words = tagger(text)
    word_list = [word.surface for word in words if re.match(r'\w', word.surface)]
    return words, word_list


def remove_english_words(word_list):
    """
    リストから英単語を削除する関数。
    
    Parameters:
    word_list (list): フィルタリングする単語のリスト。
    
    Returns:
    list: 英単語が削除された単語のリスト。
    """
    # 英単語を判定する正規表現パターン
    english_word_pattern = re.compile(r'^[a-zA-Z]+$')
    
    # 英単語ではない単語のみをリストに残す
    filtered_word_list = [word for word in word_list if not english_word_pattern.match(word)]
    
    return filtered_word_list


def filter_stop_words(word_list, stop_words):
    return [word for word in word_list if word not in stop_words]

In [12]:
import pdfplumber
import pandas as pd
import requests
import re
from fugashi import Tagger
from collections import Counter
import os



def main():
    # PDFファイルパスの入力
    pdf_path = input("PDFファイルのパスを入力してください: ")
    
    if not os.path.exists(pdf_path):
        print("指定されたPDFファイルが見つかりません。")
        return

    # DeepL APIキーの入力
    deepl_api_key = "4dfb42f0-b1c7-53d7-5e39-e255f7a461bc:fx"

    # PDFからテキストを抽出
    text = extract_text_from_pdf(pdf_path)

    # テキストを形態素解析
    tagger = Tagger()
    words, word_list = perform_morphological_analysis(text)

    # 単語の頻度を集計
    word_df = count_word_frequencies(word_list)

    # ふりがなを追加
    word_df["ふりがな"] = word_df["単語"].apply(lambda word: get_furigana(word, tagger))

    # 英訳を追加
    word_df["英訳"] = word_df["単語"].apply(lambda word: translate_to_english(word, deepl_api_key))

    # 品詞情報を追加
    word_df["品詞"] = [word.feature.split(',')[0] for word in words if re.match(r'\w', word.surface)]
    
    return word_df




if __name__ == "__main__":
    word_df = main()

word_df

KeyboardInterrupt: 

In [13]:
text = ''
with pdfplumber.open("可視化の技術2_03.pdf") as pdf:
    for page in pdf.pages:
        text += page.extract_text()

In [50]:



words, word_list = perform_morphological_analysis(text)

stop_words = set([
    "の", "に", "は", "を", "た", "が", "で", "て", "と", "し", "れ", "さ", "ある", 
    "する", "いる", "も", "こと", "から", "な", "として", "い", "や", "など", "なっ",
    "よう", "これ", "それ", "あれ", "この", "その", "あの", "ここ", "そこ", "あそこ",
    "そして", "しかし", "だから", "また", "それで", "なぜ", "なん", "どう", "何",
    "一", "二", "三", "四", "五", "六", "七", "八", "九", "十", "〇", "０", "１", "２",
    "３", "４", "５", "６", "７", "８", "９", "。", "、", "「", "」", "（", "）", "：",
    "・", "！", "？", "－", "〜", "…", "％", "＄", "＆", "＊", "＠", "＃", "＋", "＝",
    "＜", "＞", "～", "〜", "【", "】", "〔", "〕", "〝", "〟", "〃", "〓", "※", "々",
    "〆", "〤", "ヶ", "ヵ", "㍉", "㌔", "㌢", "㍍", "㌘", "㌧", "㌃", "㌶", "㍑", "㍗",
    "㌍", "㌦", "㌣", "㌫", "㍊", "㌻", "ヶ所", "ヶ月", "ヶ国", "0", "1", "2", "3", "4",
    "5", "6", "7", "8", "9", "-", "―", "ー", "＝", "〜", "‖", "｜", "／", "＼", "〜", "〆",
    "_"
])


word_list = filter_stop_words(word_list=word_list,stop_words=stop_words)

# 使用例
filtered_words = remove_english_words(word_list=word_list)

In [52]:
def count_word_frequencies(word_list):
    word_counts = Counter(word_list)
    word_df = pd.DataFrame(word_counts.items(), columns=["単語", "頻度"]).sort_values(by="頻度", ascending=False)
    return word_df


word_df = count_word_frequencies(word_list=filtered_words)

In [56]:
import itertools 

In [59]:
[word[1][0] for word in word_df.iterrows()]

C:\Users\ranza\AppData\Local\Temp\ipykernel_2832\2236902337.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [word[1][0] for word in word_df.iterrows()]


['密度',
 'プロット',
 'データ',
 '_',
 '可視',
 '化',
 'グラフ',
 '分布',
 'ヒストグラム',
 '作成',
 'カーネル',
 'ライン',
 '推定',
 '各',
 'リッジ',
 '年',
 'られる',
 'よる',
 '10',
 '技術',
 '型',
 'れる',
 '点',
 '方法',
 '確率',
 '表現',
 '関数',
 '項目',
 '追加',
 '対象',
 '集計',
 '131',
 '描画',
 '用い',
 'ライブラリ',
 '気温',
 '平均',
 '日',
 'ごと',
 'ファイル',
 '20',
 '値',
 'ロング',
 '含ま',
 'ビン',
 '標本',
 '図',
 '呼ば',
 '中心',
 'みよう',
 'ら',
 '得',
 'ため',
 'とき',
 'あり',
 'バンド',
 '幅',
 'ワイド',
 '計算',
 '分',
 '映画',
 '場合',
 '階級',
 'つい',
 '簡単',
 'オプション',
 '手法',
 'たり',
 '範囲',
 '30',
 '500',
 '軸',
 'しよう',
 '用意',
 '縦軸',
 '得点',
 '形状',
 '問題',
 'よっ',
 '線',
 '演習',
 '期末',
 '試験',
 '影響',
 'ない',
 '表',
 'サイズ',
 'いう',
 '挙げ',
 '数',
 'インストール',
 'できる',
 '並べ',
 '用いる',
 '気象',
 'もの',
 '差異',
 '間',
 'ン',
 '複数',
 'セット',
 'ジョイ',
 'リリース',
 'ジャンル',
 '塗りつぶし',
 '形式',
 '囲ま',
 '長',
 '評価',
 '科目',
 '点数',
 '保存',
 'サイト',
 '庁',
 'クリーニング',
 '描く',
 'オプショ',
 '比較',
 '入手',
 '余計',
 '情報',
 'データフレ',
 '太',
 '月',
 'ーム',
 '指定',
 '読み込み',
 '準備',
 '那覇',
 '望ましい',
 '変更',
 '引数',
 'タイトル',
 '好き',
 '地点',
 '適用',
 'デ',
 'ータ'